![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [96]:
import pandas as pd
import numpy as np

# Start coding here...

In [97]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [98]:
client_cols = ["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]
campaign_cols =  ["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome", "last_contact_date"]
economics_cols = ["client_id", "cons_price_idx", "euribor_three_months"]

In [99]:
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')

In [100]:
# Check if 'last_contact_date' is in the DataFrame columns before copying
client_cols = [col for col in client_cols if col in df.columns]
campaign_cols = [col for col in campaign_cols if col in df.columns]
economics_cols = [col for col in economics_cols if col in df.columns]

client = df[client_cols].copy()
campaign = df[campaign_cols].copy()
economics = df[economics_cols].copy()




In [101]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,nonexistent,no
1,1,1,149,0,nonexistent,no
2,2,1,226,0,nonexistent,no
3,3,1,151,0,nonexistent,no
4,4,1,307,0,nonexistent,no


In [102]:
#client -> job column Change "." to "_"
client['job']=client['job'].str.replace('.','_')




client['job'].head()

0    housemaid
1     services
2     services
3       admin_
4     services
Name: job, dtype: object

In [103]:
#client -> education column Change "." to "_" and "unknown" to np.NaN
client['education']=client['education'].str.replace('.','_')
client['education']=client['education'].replace("unknown", np.NaN)
client['education'].head()



0       basic_4y
1    high_school
2    high_school
3       basic_6y
4    high_school
Name: education, dtype: object

In [104]:
client['credit_default'].value_counts()

no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64

In [105]:
#client -> credit_default column Convert to boolean data type:1 if "yes", otherwise 0

client['credit_default'] = (client['credit_default'] == 'yes').astype(bool)

print(client['credit_default'].value_counts())


False    41185
True         3
Name: credit_default, dtype: int64


In [106]:
client['credit_default'].dtype

dtype('bool')

In [107]:
client['mortgage'].value_counts()

yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64

In [108]:
#client -> mortgage column Convert to boolean data type: 1 if "yes", otherwise 0
client['mortgage'] = (client['mortgage'] == 'yes').astype(bool)
print(client['mortgage'].value_counts())

True     21576
False    19612
Name: mortgage, dtype: int64


In [109]:
campaign['previous_outcome'] .value_counts()

nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64

In [110]:
#campaign -> previous_outcome column covert to boolean data type: 1 is "success" , otherwise 0
campaign['previous_outcome'] = (campaign['previous_outcome']  == 'success').astype(bool)
print(campaign['previous_outcome'] .value_counts())

False    39815
True      1373
Name: previous_outcome, dtype: int64


In [111]:
campaign['campaign_outcome'] .value_counts()

no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [112]:
#campaign -> campaign_outcome column covert to boolean data type: 1 is "yes" , otherwise 0
campaign['campaign_outcome']= (campaign['campaign_outcome']  == 'yes').astype(bool)
campaign['campaign_outcome'] .value_counts()

False    36548
True      4640
Name: campaign_outcome, dtype: int64

In [113]:
campaign.columns

Index(['client_id', 'number_contacts', 'contact_duration',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome'],
      dtype='object')

In [114]:
client.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage'],
      dtype='object')

In [115]:
economics.columns

Index(['client_id', 'cons_price_idx', 'euribor_three_months'], dtype='object')

In [116]:
df['month'].value_counts()

may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64

In [117]:
#campaign -> last_contact_date Create from a combination of day, month, and a newly created year column (which should have a value of 2022); Format = "YYYY-MM-DD"
from datetime import datetime
df['year'] = 2022
df['month_new'] = df['month'].astype(str).apply(lambda x: datetime.strptime(x.strip(), '%b').month)

# Create a new date column in YYYY-MM-DD format
campaign['last_contact_date'] = pd.to_datetime(dict(year=df['year'], month=df['month_new'], day=df['day']))
print(campaign['last_contact_date'].head())
print(campaign['last_contact_date'].dtype)  



0   2022-05-13
1   2022-05-19
2   2022-05-23
3   2022-05-27
4   2022-05-03
Name: last_contact_date, dtype: datetime64[ns]
datetime64[ns]


In [118]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)